# Coordinates in SunPy

## Stuart Mumford


<a href="http://shef.ac.uk">
<img style='float: left; width: 30%; margin-top: 90px; height: 25%;' src='images/TUOS_Logo_CMYK_Keyline.svg'/>
</a>
<a href="https://aperiosoftware.com">
<img style='float: right; width: 30%; margin-top: 90px; height: 25%;' src='images/aperio_logo.svg'/>
</a>
<a href="http://sunpy.org">
<img style='float: right; width: 30%; margin-top: 90px; height: 25%; margin-right: 5%;' src='images/sunpy_logo.svg'/></a>

# Coordinates?

- Pixel (Data) Coordinates
- World Coordinates
- Physical Coordinates

In [3]:
import astropy.units as u

from astropy.coordinates import SkyCoord
import sunpy.coordinates

In [6]:
sc = SkyCoord(100*u.arcsec, 0.1*u.deg, frame='helioprojective')
sc

<SkyCoord (Helioprojective: dateobs=None, rsun=695508.0 km, observer=<HeliographicStonyhurst Coordinate (dateobs=None): (lon, lat, radius) in (deg, deg, AU)
    ( 0.,  0.,  1.)>): (Tx, Ty) in arcsec
    ( 100.,  360.)>

In [7]:
sc.transform_to('heliographic_stonyhurst')

<SkyCoord (HeliographicStonyhurst: dateobs=None): (lon, lat, radius) in (deg, deg, km)
    ( 6.42759277,  21.94999839,  695507.99999766)>